In [ ]:
import pandas as pd
from citipy import citipy
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
latitude = []
longitude = []

for each_lat in np.random.randint(-90,90,2000):
    latitude.append(each_lat)

for each_lon in np.random.randint(-180,180,2000):
    longitude.append(each_lon)
    
latitude_and_Longitude = tuple(zip(latitude,longitude))

    

In [ ]:
cities = []
country = []

for lat,lon in latitude_and_Longitude:
    city = citipy.nearest_city(lat,lon)
    
    cityName = city.city_name
    
    cities.append(cityName)
    
    country.append(city.country_code)

In [ ]:
print('The above script has generated {} different cities using random numbers.'.format(len(set(cities))))

#create a dataframe to store all the cities and countries generated with citipy

df_countries = pd.DataFrame({'Cities': cities,
              'Country': country})

#add additional blank columns to store information from openweathermap api

df_countries['Latitude'] = ''
df_countries['Longitude'] = ''
df_countries['Temperature (F)'] = ''
df_countries['Humidity (%)'] = ''
df_countries['Cloudiness (%)'] = ''
df_countries['Wind Speed (mph)'] = ''

In [ ]:
df_countries = df_countries.drop_duplicates(subset=['Cities'],keep='first')


api_key = "5be78fade1727ace968b5ab363d997bd"


for index,row in df_countries.iterrows():
    city_name = row['Cities']
    Country_id = row['Country']
    
    
    #api call for all Cities 
    api_url = "http://api.openweathermap.org/data/2.5/forecast" \
    "?q={},{}&units=IMPERIAL&mode=json&APPID={}".format(city_name,Country_id,api_key)
    
    
    country_info = requests.get(api_url).json()
    
    #print all api url's for each city
    print(api_url)
    
    
    try:
        df_countries.set_value(index,'Latitude',country_info['city']['coord']['lat'])
        df_countries.set_value(index,'Longitude',country_info['city']['coord']['lon'])
        df_countries.set_value(index,'Temperature (F)',country_info['list'][0]['main']['temp'])
        df_countries.set_value(index,'Humidity (%)',country_info['list'][0]['main']['humidity'])
        df_countries.set_value(index,'Cloudiness (%)',country_info['list'][0]['clouds']['all'])
        df_countries.set_value(index,'Wind Speed (mph)',country_info['list'][0]['wind']['speed'])
        
    except KeyError:
        df_countries.set_value(index,'Latitude',np.nan)
        df_countries.set_value(index,'Longitude',np.nan)
        df_countries.set_value(index,'Temperature (F)',np.nan)
        df_countries.set_value(index,'Humidity (%)',np.nan)
        df_countries.set_value(index,'Cloudiness (%)',np.nan)
        df_countries.set_value(index,'Wind Speed (mph)',np.nan)
        
        print('Missing weather information...skip')


In [ ]:
df_countries['Latitude'] = pd.to_numeric(df_countries['Latitude'])
df_countries['Longitude'] = pd.to_numeric(df_countries['Longitude'])
df_countries['Temperature (F)'] = pd.to_numeric(df_countries['Temperature (F)'])
df_countries['Humidity (%)'] = pd.to_numeric(df_countries['Humidity (%)'])
df_countries['Cloudiness (%)'] = pd.to_numeric(df_countries['Cloudiness (%)'])
df_countries['Wind Speed (mph)'] = pd.to_numeric(df_countries['Wind Speed (mph)'])

In [ ]:
df_countries.head()

In [ ]:
<style> .dataframe thead tr:only-child th { text-align: right; }
.dataframe thead th {
    text-align: left;
}

.dataframe tbody tr th {
    vertical-align: top;
}
</style>

In [ ]:
length_of_original_DF = len(df_countries)
length_after_dropna = len(df_countries.dropna())

missing_weather_info = length_of_original_DF - length_after_dropna


print('For the following cities associated with this dataframe, {} cities did not contain weather\
 information. These cities\t will be dropped from this dataframe.'.format(missing_weather_info))

print('\n')

print("The dataframe used for plotting contains weather information for {} different cities. "\
      .format(len(df_countries.dropna())))

In [ ]:
df_countries = df_countries.dropna()
df_countries.to_csv(path_or_buf='df_countries.csv')

In [ ]:
df_countries_table = df_countries.copy()
df_countries_table['Latitude'] = pd.qcut(df_countries['Latitude'],11,precision=0)

In [ ]:
cm = sns.light_palette('green',as_cmap=True)

df_countries_table.groupby(['Latitude'])['Temperature (F)'].mean().reset_index().style.background_gradient(cmap=cm)

<style type="text/css" > #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow0_col1 { background-color: #3ba13b; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow1_col1 { background-color: #078407; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow2_col1 { background-color: #068306; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow3_col1 { background-color: #008000; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow4_col1 { background-color: #028102; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow5_col1 { background-color: #148b14; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow6_col1 { background-color: #46a746; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow7_col1 { background-color: #76c176; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow8_col1 { background-color: #a5dba5; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow9_col1 { background-color: #b9e7b9; } #T_80b904f0_ccd9_11e7_b5c0_a45e60bc420drow10_col1 { background-color: #e5ffe5; }</style>

In [ ]:
plt.style.use('dark_background')


In [ ]:
plt.figure(figsize=(18,5))
plt.scatter(df_countries['Latitude'],df_countries['Temperature (F)'],color = 'red',s=20)
plt.title('Temperature (F) vs. Latitude',fontsize=(18));
plt.xlim(-90,90)
plt.ylabel('Temperature (F)',fontsize=(15));
plt.xlabel('Latitude',fontsize=(15));

In [ ]:
df_countries_table.groupby(['Latitude'])['Humidity (%)'].mean().reset_index().style.background_gradient(cmap=cm)
<style type="text/css" > #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow0_col1 { background-color: #aadeaa; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow1_col1 { background-color: #e5ffe5; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow2_col1 { background-color: #168c16; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow3_col1 { background-color: #008000; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow4_col1 { background-color: #49a849; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow5_col1 { background-color: #9ed89e; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow6_col1 { background-color: #8dce8d; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow7_col1 { background-color: #98d498; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow8_col1 { background-color: #5eb45e; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow9_col1 { background-color: #319b31; } #T_88f2b918_ccd9_11e7_9b73_a45e60bc420drow10_col1 { background-color: #6dbc6d; }</style>

In [ ]:
plt.figure(figsize=(18,5))
plt.scatter(df_countries['Latitude'],df_countries['Humidity (%)'],color = 'white',s=20);
plt.title('Humidity (%) vs. Latitude',fontsize=(18));
plt.xlim(-90,90)
plt.ylabel('Humidity (%)',fontsize=(15));
plt.xlabel('Latitude',fontsize=(15));

In [ ]:
df_countries_table.groupby(['Latitude'])['Cloudiness (%)'].mean().reset_index().style.background_gradient(cmap=cm)
<style type="text/css" > #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow0_col1 { background-color: #d4f6d4; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow1_col1 { background-color: #9fd89f; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow2_col1 { background-color: #44a644; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow3_col1 { background-color: #8bcd8b; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow4_col1 { background-color: #52ad52; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow5_col1 { background-color: #acdfac; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow6_col1 { background-color: #e5ffe5; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow7_col1 { background-color: #97d497; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow8_col1 { background-color: #79c379; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow9_col1 { background-color: #008000; } #T_8ac39230_ccd9_11e7_ad5e_a45e60bc420drow10_col1 { background-color: #44a544; }</style>

In [ ]:
plt.figure(figsize=(18,5))
plt.scatter(df_countries['Latitude'],df_countries['Cloudiness (%)'],color = 'w',s=20);
plt.title('Cloudiness (%) vs. Latitude',fontsize=(18));
plt.xlim(-90,90)
plt.ylabel('Cloudiness (%)',fontsize=(15));
plt.xlabel('Latitude',fontsize=(15));

In [ ]:
df_countries_table.groupby(['Latitude'])['Wind Speed (mph)'].mean().reset_index().style.background_gradient(cmap=cm)
<style type="text/css" > #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow0_col1 { background-color: #008000; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow1_col1 { background-color: #098509; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow2_col1 { background-color: #9cd69c; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow3_col1 { background-color: #dcfadc; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow4_col1 { background-color: #dcfadc; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow5_col1 { background-color: #e5ffe5; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow6_col1 { background-color: #7cc57c; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow7_col1 { background-color: #038103; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow8_col1 { background-color: #1d901d; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow9_col1 { background-color: #229322; } #T_8e7526c8_ccd9_11e7_b919_a45e60bc420drow10_col1 { background-color: #5eb45e; }</style>

In [ ]:
plt.figure(figsize=(18,5))
plt.scatter(df_countries['Latitude'],df_countries['Wind Speed (mph)'],color = 'red',s=20);
plt.xlim(-90,90)
plt.title('Wind Speed (mph) vs. Latitude',fontsize=(18));
plt.ylabel('Temperature Wind Speed (mph)',fontsize=(15));
plt.xlabel('Latitude',fontsize=(15));

In [ ]:
print("The average wind speed is {} mph for all randomly selected cities".format(df_countries['Wind Speed (mph)'].mean()))
The average wind speed is 8.35529318541997 mph for all randomly selected cities


In [ ]:
#The wind speeds are higher on average at the north and south poles.

#In the Wind Speed (mph) vs. Latitude scatter plot, the average when speed for all cities is fairly low.

#In the Temperature vs. Latitude scatter plot, as the latitude increases the temperature increases.
